<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/Merge_a_LoRA_Adapter_and_Export_to_ONNX_Example_with_Phi_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*More details in this article: [transformers.js: Run Phi-3.5 & Llama 3.2 in Your Web Browser](https://newsletter.kaitchup.com/p/transformersjs-run-phi-35-and-llama)*

This notebook shows how to do if you have an LoRA adapter that you want to export to ONNX. Adapters can't be exported to ONNX directly, they must be merged first into the base model, then the resulting model can be exported.

#Installation

In [ ]:
!pip install --upgrade transformers peft optimum[onnxruntime]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Success

#Merge the Adapter

In [ ]:
import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

model_name = "microsoft/Phi-3.5-mini-instruct"
adapter = "./Phi-3.5-Mini-LoRA" #Your LoRA adapter fine-tuned
output_dir = "./Phi-3.5-Mini-LoRA-Merge"

#Load the model
model = AutoModelForCausalLM.from_pretrained(
          model_name,  device_map={"": 0}, torch_dtype=torch.bfloat16
)

#Load the adapter
model = PeftModel.from_pretrained(model, adapter)

#Merge the adapter
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name)

#Save the new model and its tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

#ONNX Export

In [ ]:
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer

model_checkpoint = "./Phi-3.5-Mini-LoRA-Merge"
save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForCausalLM.from_pretrained(model_checkpoint, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Framework not specified. Using pt to export the model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: Phi3ForCausalLM *****
Using framework PyTorch: 2.4.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> True
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/transformers/models/phi3/modeling_phi3.py:96: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
You are not running the flash-attention implementation, expect numerical differences.
/usr/local/lib/python3.10/dist-packag

('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/tokenizer.model',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')